In [1]:
# do pip installs to get libraries: wordfreq, googletrans, editdistance, unidecode

from wordfreq import word_frequency
from googletrans import Translator
import editdistance
from unidecode import unidecode

import time

translator = Translator()

This script will process the data files to add information about translations and frequencies.

In [14]:
lang='fr_en'
fn = 'data/data_{0}/{0}.slam.20171218.train.new'.format(lang)
(src, dest)=lang.split('_')
print("Translating %s to %s" %(src,dest))


Translating fr to en


In [15]:
fn

'data/data_fr_en/fr_en.slam.20171218.train.new'

In [16]:
#Todd's code
# trans_cache = {}

# newlines=[]
# with open(fn, 'r') as f:
#     for line in f:
#         if len(line)>1:
#             #print("***"+line)
#             if line[0]!='#':
#                 # process and translate
#                 pieces = line.split()
#                 idx = pieces[0]
#                 word = pieces[1]
#                 root_token = pieces[2]
#                 part_of_speech = pieces[3]
#                 morph_feats = pieces[4]
#                 dependency_label = pieces[5]
#                 dependency_edge_head = pieces[6]
                
#                 clean_word = word.lower()
#                 if '\'' not in clean_word:
#                     if clean_word not in trans_cache:
#                         translation_obj = translator.translate(clean_word,src=src,dest=dest)
#                         translation = translation_obj.text
#                         print("%s to %s" % (clean_word, translation))  # need to do some clean up of the translation
#                         trans_cache[clean_word]=translation
#                     else:
#                         translation = trans_cache[clean_word]
                
#                 trans_freq = word_frequency(translation, dest)
#                 src_frec = word_frequency(word, src)
                
                
#                 if len(pieces) == 8:
#                     label = pieces[7]
#                     repl = [idx, word, root_token, part_of_speech, morph_feats, dependency_label, dependency_edge_head, label]
#                 else:
#                     repl = [idx, word, root_token, part_of_speech, morph_feats, dependency_label, dependency_edge_head]
#                 newlines.append("\t".join(repl)+'\n')
#             else:
#                 newlines.append(line)
#         else:
#             newlines.append(line)

# print(len(newlines))
# with open('test.txt','w') as fp:
#     fp.writelines(newlines)

In [17]:
#Pam's code
# FOR EVERY ROOT WORD (e.g, "be" instead of "is")
# Make csv of translation and calculate a simple metric of how much of a "cognate" it is

#output filename
fnout = "{}_{}_rootwordfeats.txt".format(src,dest)
print("Will save translations and features to: ",fnout)


trans_cache = {}

newlines=[]

with open(fn, 'r') as f:
    for line in f:
        if len(line)>1 and line[0]!='#':
            # process and translate
            pieces = line.split()
            idx = pieces[0]
            word = pieces[1]
            root_token = pieces[2]
            part_of_speech = pieces[3]
            morph_feats = pieces[4]
            dependency_label = pieces[5]
            dependency_edge_head = pieces[6]

            clean_word = root_token.lower() # Only using root token here rather than word
            if '\'' not in clean_word and clean_word not in trans_cache:
                # Wait so as not to annoy the API (otherwise will give JSON error)
                time.sleep(0.2)

                # Translate so we know the word in the language the user already knows
                translation_obj = translator.translate(clean_word,src=src,dest=dest)
                translation = translation_obj.text

                # Clean up translation: 
                # remove accents, make lowercase, and remove any non-informative first words
                clean_trans = unidecode(translation)
                clean_trans = clean_trans.lower() 
                g = clean_trans.split(" ")
                if len(g) == 2:
                    if g[0] in ['to','i','we','you','they','he','she','the',"i'll"]:
                        clean_trans = g[1]          
                elif len(g) > 2:
                    if g[1] in ['will']:
                        clean_trans = ' '.join(g[2:len(g)])

                # levenshtein distance features
                levdist = editdistance.eval(clean_word, clean_trans)
                levdistfrac = levdist/max(len(clean_word),len(clean_trans))
                
                # print to console translation and LD 
                print("%s to %s (LD=%i, LDfrac=%.2f)" % (clean_word, clean_trans, levdist, levdistfrac))
                trans_cache[clean_word]=translation

                # word frequency of translation (known language) and src words
                trans_freq = word_frequency(translation, dest)
                src_freq = word_frequency(root_token, src)

                # write to csv file
                newrow = [root_token, clean_trans, str(src_freq), str(levdist), str(levdistfrac)]
                newlines.append(",".join(newrow)+'\n')

                
print(len(newlines))
with open(fnout,'w') as fp:
    fp.writelines(newlines)

Will save translations and features to:  fr_en_rootwordfeats.txt
le to the (LD=2, LDfrac=0.67)
garçon to boy (LD=5, LDfrac=0.83)
il to he (LD=2, LDfrac=1.00)
suivre to follow (LD=6, LDfrac=1.00)
un to a (LD=2, LDfrac=1.00)
femme to wife (LD=4, LDfrac=0.80)
fille to girl (LD=3, LDfrac=0.60)
riche to rich (LD=1, LDfrac=0.20)
rouge to red (LD=4, LDfrac=0.80)
homme to man (LD=4, LDfrac=0.80)
être to be (LD=3, LDfrac=0.75)
pomme to apple (LD=4, LDfrac=0.80)
taire to shut up (LD=7, LDfrac=1.00)
manger to eat (LD=5, LDfrac=0.83)
lui to him (LD=3, LDfrac=1.00)
calme to calm (LD=1, LDfrac=0.20)
avoir to have (LD=4, LDfrac=0.80)
enfant to child (LD=6, LDfrac=1.00)
et to and (LD=3, LDfrac=1.00)
robe to dress (LD=4, LDfrac=0.80)
chat to cat (LD=1, LDfrac=0.25)
noir to black (LD=5, LDfrac=1.00)
se to himself (LD=5, LDfrac=0.71)
sommer to summon (LD=3, LDfrac=0.50)
écrire to write (LD=3, LDfrac=0.50)
aimer to like (LD=3, LDfrac=0.60)
calmer to calm (LD=2, LDfrac=0.33)
journal to newspaper (LD=8, LDf

en to in (LD=1, LDfrac=0.50)
octobre to october (LD=2, LDfrac=0.29)
septembre to september (LD=2, LDfrac=0.22)
décembre to december (LD=3, LDfrac=0.38)
juin to june (LD=2, LDfrac=0.50)
minuit to midnight (LD=4, LDfrac=0.50)
deux to two (LD=4, LDfrac=1.00)
minute to minute (LD=0, LDfrac=0.00)
seize to sixteen (LD=5, LDfrac=0.71)
seconde to second (LD=1, LDfrac=0.14)
durer to last (LD=5, LDfrac=1.00)
difficile to difficult (LD=2, LDfrac=0.22)
dire to say (LD=4, LDfrac=1.00)
déjeuner to lunch (LD=6, LDfrac=0.75)
vendre to sale (LD=5, LDfrac=0.83)
marcher to walk (LD=6, LDfrac=0.86)
griller to grill (LD=2, LDfrac=0.29)
inviter to invite (LD=1, LDfrac=0.14)
famille to family (LD=2, LDfrac=0.29)
là to the (LD=3, LDfrac=1.00)
très to very (LD=4, LDfrac=1.00)
che to che (LD=0, LDfrac=0.00)
f to f (LD=0, LDfrac=0.00)
jamais to never (LD=6, LDfrac=1.00)
personne to no one (LD=5, LDfrac=0.62)
parler to speak (LD=6, LDfrac=1.00)
comprendre to understand (LD=9, LDfrac=0.90)
pourquoi to why (LD=8, L

vert to green (LD=4, LDfrac=0.80)
loin to far (LD=4, LDfrac=1.00)
oeil to eye (LD=3, LDfrac=0.75)
cœur to heart (LD=4, LDfrac=0.80)
tenir to hold (LD=5, LDfrac=1.00)
langue to language (LD=2, LDfrac=0.25)
oeuf to egg (LD=3, LDfrac=0.75)
finir to finish (LD=2, LDfrac=0.33)
bonne to good (LD=4, LDfrac=0.80)
pauvre to poor (LD=4, LDfrac=0.67)
fort to strong (LD=5, LDfrac=0.83)
laid to ugly (LD=4, LDfrac=1.00)
italien to italian (LD=1, LDfrac=0.14)
sympa to friendly (LD=8, LDfrac=1.00)
africain to african (LD=1, LDfrac=0.12)
sérieux to serious (LD=3, LDfrac=0.43)
prête to ready (LD=5, LDfrac=1.00)
gentil to nice (LD=5, LDfrac=0.83)
allemand to german (LD=5, LDfrac=0.62)
sale to dirty (LD=5, LDfrac=1.00)
propre to clean (LD=6, LDfrac=1.00)
cinq to five (LD=3, LDfrac=0.75)
six to six (LD=0, LDfrac=0.00)
neuf to new (LD=2, LDfrac=0.50)
nombre to number (LD=3, LDfrac=0.50)
douze to twelve (LD=5, LDfrac=0.83)
vingt to twenty (LD=5, LDfrac=0.83)
prénom to first name (LD=7, LDfrac=0.70)
clair to 

linge to washing (LD=5, LDfrac=0.71)
radio to radio (LD=0, LDfrac=0.00)
caisse to checkout (LD=7, LDfrac=0.88)
écran to screen (LD=3, LDfrac=0.50)
plateau to tray (LD=5, LDfrac=0.71)
clavier to keyboard (LD=7, LDfrac=0.88)
logique to logic (LD=3, LDfrac=0.43)
pire to worst (LD=4, LDfrac=0.80)
indispensable to essential (LD=9, LDfrac=0.69)
aucun to no (LD=5, LDfrac=1.00)
ordinaire to ordinary (LD=2, LDfrac=0.22)
inutile to useless (LD=6, LDfrac=0.86)
entre to enter (LD=2, LDfrac=0.40)
chine to china (LD=1, LDfrac=0.20)
bordeaux to bordeaux (LD=0, LDfrac=0.00)
france to la france (LD=3, LDfrac=0.33)
village to town (LD=7, LDfrac=1.00)
garage to garage (LD=0, LDfrac=0.00)
ville to city (LD=4, LDfrac=0.80)
peser to weigh (LD=4, LDfrac=0.80)
travailler to work (LD=10, LDfrac=1.00)
exister to exist (LD=2, LDfrac=0.29)
priser to prize (LD=2, LDfrac=0.33)
souvenir to memory (LD=7, LDfrac=0.88)
continuer to carry on (LD=8, LDfrac=0.89)
servir to serve (LD=2, LDfrac=0.33)
douter to doubt (LD=3, 

rendre to return (LD=3, LDfrac=0.50)
pari to bet (LD=4, LDfrac=1.00)
près to near (LD=4, LDfrac=1.00)
rarement to rarely (LD=4, LDfrac=0.50)
suffisamment to enough (LD=12, LDfrac=1.00)
gros to large (LD=4, LDfrac=0.80)
souhaiter to wish (LD=8, LDfrac=0.89)
européen to european (LD=2, LDfrac=0.25)
couvert to covered (LD=3, LDfrac=0.43)
cuillère to spoon (LD=8, LDfrac=1.00)
japonais to japanese (LD=4, LDfrac=0.50)
ver to worm (LD=3, LDfrac=0.75)
plusieurs to many (LD=9, LDfrac=1.00)
privée to private (LD=2, LDfrac=0.29)
gras to fat (LD=3, LDfrac=0.75)
certain to certain (LD=0, LDfrac=0.00)
reine to queen (LD=4, LDfrac=0.80)
propriété to property (LD=4, LDfrac=0.44)
plage to beach (LD=4, LDfrac=0.80)
compagnie to company (LD=3, LDfrac=0.33)
entreprendre to undertake (LD=8, LDfrac=0.67)
préparer to prepare (LD=2, LDfrac=0.25)
revenir to come back (LD=8, LDfrac=0.89)
jardin to garden (LD=2, LDfrac=0.33)
toucher to touch (LD=2, LDfrac=0.29)
découvrir to discover (LD=5, LDfrac=0.56)
compter t

vol to flight (LD=6, LDfrac=1.00)
ok to ok (LD=0, LDfrac=0.00)
cuire to cook (LD=4, LDfrac=0.80)
larme to tear (LD=4, LDfrac=0.80)
aussi to also (LD=3, LDfrac=0.60)
coin to corner (LD=3, LDfrac=0.50)
profond to deep (LD=7, LDfrac=1.00)
claire to clear (LD=3, LDfrac=0.50)
honneur to honor (LD=3, LDfrac=0.43)
sen to sen (LD=0, LDfrac=0.00)
paix to peace (LD=3, LDfrac=0.60)
belle to beautiful (LD=7, LDfrac=0.78)
mince to slim (LD=5, LDfrac=1.00)
national to national (LD=0, LDfrac=0.00)
sympathique to nice (LD=9, LDfrac=0.82)
séjourner to stay (LD=8, LDfrac=0.89)
nuit to night (LD=3, LDfrac=0.60)
autre to other (LD=4, LDfrac=0.80)
garder to keep (LD=5, LDfrac=0.83)
quitter to leave (LD=6, LDfrac=0.86)
message to message (LD=0, LDfrac=0.00)
argument to argument (LD=0, LDfrac=0.00)
dent to tooth (LD=4, LDfrac=0.80)
sec to dry (LD=3, LDfrac=1.00)
par to by (LD=3, LDfrac=1.00)
lourde to heavy (LD=6, LDfrac=1.00)
époux to husband (LD=7, LDfrac=1.00)
stupide to dumb (LD=6, LDfrac=0.86)
médecin t

aéroport to airport (LD=2, LDfrac=0.25)
international to international (LD=0, LDfrac=0.00)
moteur to engine (LD=6, LDfrac=1.00)
malade to sick (LD=6, LDfrac=1.00)
offre to offer (LD=2, LDfrac=0.40)
syndicat to union (LD=6, LDfrac=0.75)
marché to market (LD=3, LDfrac=0.50)
tennis to tennis (LD=0, LDfrac=0.00)
nager to swim (LD=5, LDfrac=1.00)
compétition to competition (LD=1, LDfrac=0.09)
louer to rent (LD=5, LDfrac=1.00)
euro to euro (LD=0, LDfrac=0.00)
crédit to credit (LD=1, LDfrac=0.17)
propriétaire to owner (LD=10, LDfrac=0.83)
appartement to apartment (LD=2, LDfrac=0.18)
fantôme to ghost (LD=7, LDfrac=1.00)
saint to saint (LD=0, LDfrac=0.00)
calme. to calm. (LD=1, LDfrac=0.17)
manque to lack (LD=5, LDfrac=0.83)
chargeur to charger (LD=1, LDfrac=0.12)
pétrin to mess (LD=6, LDfrac=1.00)
grive to thrush (LD=5, LDfrac=0.83)
merle to blackbird (LD=9, LDfrac=1.00)
utilison to utilison (LD=0, LDfrac=0.00)
pénétrer to enter (LD=4, LDfrac=0.50)
excuser to excuse (LD=1, LDfrac=0.14)
état to

In [18]:
# FOR EVERY WORD (NOT root word)
# Make csv of translation and calculate a simple metric of how much of a "cognate" it is

#output filename
fnout = "{}_{}_wordwordfeats.txt".format(src,dest)
print("Will save translations and features to: ",fnout)


trans_cache = {}

newlines=[]

with open(fn, 'r') as f:
    for line in f:
        if len(line)>1 and line[0]!='#':
            # process and translate
            pieces = line.split()
            idx = pieces[0]
            word = pieces[1]
            root_token = pieces[2]
            part_of_speech = pieces[3]
            morph_feats = pieces[4]
            dependency_label = pieces[5]
            dependency_edge_head = pieces[6]

            clean_word = word.lower() # Using WORD not root 
            if '\'' not in clean_word and clean_word not in trans_cache:
                # Wait so as not to annoy the API (otherwise will give JSON error)
                time.sleep(0.2)

                # Translate so we know the word in the language the user already knows
                translation_obj = translator.translate(clean_word,src=src,dest=dest)
                translation = translation_obj.text

                # Clean up translation: 
                # remove accents, make lowercase, and remove any non-informative first words
                clean_trans = unidecode(translation)
                clean_trans = clean_trans.lower() 
                g = clean_trans.split(" ")
                if len(g) == 2:
                    if g[0] in ['to','i','we','you','they','he','she','the']:
                        clean_trans = g[1]          
                elif len(g) > 2:
                    if g[1] in ['will']: # e.g. comere, i will eat
                        clean_trans = ' '.join(g[2:len(g)])

                # levenshtein distance features
                levdist = editdistance.eval(clean_word, clean_trans)
                levdistfrac = levdist/max(len(clean_word),len(clean_trans))
                
                # print to console translation and LD 
                print("%s to %s (LD=%i, LDfrac=%.2f)" % (clean_word, clean_trans, levdist, levdistfrac))
                trans_cache[clean_word]=translation

                # word frequency of translation (known language) and src words
                trans_freq = word_frequency(translation, dest)
                src_freq = word_frequency(word, src)

                # write to csv file
                newrow = [word, clean_trans, str(src_freq), str(levdist), str(levdistfrac)]
                newlines.append(",".join(newrow)+'\n')

                
print(len(newlines))
with open(fnout,'w') as fp:
    fp.writelines(newlines)

Will save translations and features to:  fr_en_wordwordfeats.txt
le to the (LD=2, LDfrac=0.67)
garçon to boy (LD=5, LDfrac=0.83)
je to i (LD=2, LDfrac=1.00)
suis to am (LD=4, LDfrac=1.00)
une to a (LD=3, LDfrac=1.00)
femme to wife (LD=4, LDfrac=0.80)
la to the (LD=3, LDfrac=1.00)
fille to girl (LD=3, LDfrac=0.60)
un to a (LD=2, LDfrac=1.00)
riche to rich (LD=1, LDfrac=0.20)
rouge to red (LD=4, LDfrac=0.80)
homme to man (LD=4, LDfrac=0.80)
est to east (LD=1, LDfrac=0.25)
pomme to apple (LD=4, LDfrac=0.80)
tu to you (LD=2, LDfrac=0.67)
es to es (LD=0, LDfrac=0.00)
mange to eat (LD=4, LDfrac=0.80)
il to he (LD=2, LDfrac=1.00)
elle to she (LD=3, LDfrac=0.75)
calme to calm (LD=1, LDfrac=0.20)
a to at (LD=1, LDfrac=0.50)
enfant to child (LD=6, LDfrac=1.00)
et to and (LD=3, LDfrac=1.00)
robe to dress (LD=4, LDfrac=0.80)
chat to cat (LD=1, LDfrac=0.25)
noir to black (LD=5, LDfrac=1.00)
les to the (LD=3, LDfrac=1.00)
hommes to men (LD=4, LDfrac=0.67)
nous to we (LD=4, LDfrac=1.00)
sommes to are

leur to their (LD=3, LDfrac=0.60)
chapeau to hat (LD=5, LDfrac=0.71)
votre to your (LD=3, LDfrac=0.60)
ses to his (LD=2, LDfrac=0.67)
mes to my (LD=2, LDfrac=0.67)
robes to dresses (LD=4, LDfrac=0.57)
pouvons to can (LD=6, LDfrac=0.86)
adore to love (LD=3, LDfrac=0.60)
lequel to which (LD=6, LDfrac=1.00)
où to or (LD=1, LDfrac=0.50)
laquelle to which (LD=8, LDfrac=1.00)
combien to how much (LD=7, LDfrac=0.88)
lesquelles to which (LD=10, LDfrac=1.00)
quoi to what (LD=4, LDfrac=1.00)
qui to who (LD=3, LDfrac=1.00)
porte to door (LD=4, LDfrac=0.80)
manteau to coat (LD=6, LDfrac=0.86)
saisis to seized (LD=4, LDfrac=0.67)
mon to my (LD=2, LDfrac=0.67)
propose to offers (LD=6, LDfrac=0.86)
améliore to improved (LD=6, LDfrac=0.75)
son to his (LD=3, LDfrac=1.00)
bête to stupid (LD=6, LDfrac=1.00)
méchant to bad (LD=6, LDfrac=0.86)
drôle to funny (LD=5, LDfrac=1.00)
aiment to like (LD=4, LDfrac=0.67)
zéro to zero (LD=1, LDfrac=0.25)
mariage to wedding (LD=6, LDfrac=0.86)
neveu to nephew (LD=3, 

sortie to exit (LD=5, LDfrac=0.83)
sud to south (LD=3, LDfrac=0.60)
quand to when (LD=4, LDfrac=0.80)
départ to departure (LD=4, LDfrac=0.44)
changé to exchange (LD=3, LDfrac=0.38)
compris to including (LD=8, LDfrac=0.89)
obtenu to got (LD=5, LDfrac=0.83)
rentrée to return (LD=4, LDfrac=0.57)
eu to had (LD=3, LDfrac=1.00)
assez to enough (LD=6, LDfrac=1.00)
tiré to drawn (LD=5, LDfrac=1.00)
chemise to shirt (LD=5, LDfrac=0.71)
commandé to controlled (LD=8, LDfrac=0.80)
ajouté to added (LD=5, LDfrac=0.83)
citrons to lemons (LD=4, LDfrac=0.57)
envoyé to sent (LD=5, LDfrac=0.83)
attendu to expected (LD=7, LDfrac=0.88)
raté to missed (LD=6, LDfrac=1.00)
dîner to having dinner (LD=9, LDfrac=0.69)
moins to less (LD=4, LDfrac=0.80)
égale to equal (LD=4, LDfrac=0.80)
quarante to forty (LD=6, LDfrac=0.75)
sept to seven (LD=3, LDfrac=0.60)
ans to years (LD=3, LDfrac=0.60)
dix to ten (LD=3, LDfrac=1.00)
trente to thirty (LD=4, LDfrac=0.67)
cinquante to fifty (LD=7, LDfrac=0.78)
cent to hundred (L

travail to job (LD=7, LDfrac=1.00)
coiffeuse to dressing table (LD=12, LDfrac=0.86)
coiffeur to hairdresser (LD=8, LDfrac=0.73)
pouvoir to power (LD=4, LDfrac=0.57)
faire to make (LD=3, LDfrac=0.60)
novembre to november (LD=2, LDfrac=0.25)
demie to half (LD=5, LDfrac=1.00)
printemps to spring (LD=6, LDfrac=0.67)
mari to husband (LD=6, LDfrac=0.86)
papa to dad (LD=3, LDfrac=0.75)
normalement to normally (LD=5, LDfrac=0.45)
tellement to so much (LD=8, LDfrac=0.89)
généralement to usually (LD=10, LDfrac=0.83)
souris to mouse (LD=4, LDfrac=0.67)
lapin. to rabbit. (LD=4, LDfrac=0.57)
jaunes to yellows (LD=6, LDfrac=0.86)
donc to so (LD=3, LDfrac=0.75)
raisins to grapes (LD=5, LDfrac=0.71)
allez to going (LD=5, LDfrac=1.00)
voulons to want to (LD=7, LDfrac=1.00)
alcool to alcohol (LD=1, LDfrac=0.14)
prêt to ready (LD=5, LDfrac=1.00)
vivez to live (LD=2, LDfrac=0.40)
désolée to sorry (LD=5, LDfrac=0.71)
ceinture to belt (LD=6, LDfrac=0.75)
blanches to white (LD=7, LDfrac=0.88)
frères to broth

resté to rest (LD=1, LDfrac=0.20)
prise to taking (LD=5, LDfrac=0.83)
souvenues to have remembered (LD=13, LDfrac=0.87)
pris to taken (LD=5, LDfrac=1.00)
continue to keep on going (LD=10, LDfrac=0.77)
sert to serves (LD=3, LDfrac=0.50)
doute to doubt (LD=2, LDfrac=0.40)
bronze to bronze (LD=0, LDfrac=0.00)
tourne to turned (LD=2, LDfrac=0.33)
repris to taken back (LD=9, LDfrac=0.90)
milliard to billion (LD=4, LDfrac=0.50)
content to happy (LD=7, LDfrac=1.00)
celui to that (LD=5, LDfrac=1.00)
préfère to prefer (LD=3, LDfrac=0.43)
poubelle to trash can (LD=9, LDfrac=1.00)
jouet to toy (LD=4, LDfrac=0.80)
efficace to effective (LD=4, LDfrac=0.44)
pratique to convenient (LD=9, LDfrac=0.90)
chambre to bedroom (LD=7, LDfrac=1.00)
terre to earth (LD=4, LDfrac=0.80)
marron to brown (LD=4, LDfrac=0.67)
cadeaux to gifts (LD=7, LDfrac=1.00)
monsieur to gentleman (LD=7, LDfrac=0.78)
culture to culture (LD=0, LDfrac=0.00)
commence to begin (LD=7, LDfrac=0.88)
explique to explain (LD=4, LDfrac=0.50)

aléatoire to random (LD=8, LDfrac=0.89)
présent to present (LD=1, LDfrac=0.14)
place to square (LD=4, LDfrac=0.67)
vide to empty (LD=5, LDfrac=1.00)
occupé to occupied (LD=3, LDfrac=0.38)
technique to technical (LD=3, LDfrac=0.33)
nécessaire to necessary (LD=3, LDfrac=0.30)
physique to physical (LD=3, LDfrac=0.38)
moderne to modern (LD=1, LDfrac=0.14)
publique to public (LD=3, LDfrac=0.38)
émotion to emotion (LD=1, LDfrac=0.14)
immense to immense (LD=0, LDfrac=0.00)
fragile to fragile (LD=0, LDfrac=0.00)
classique to classical (LD=3, LDfrac=0.33)
extrême to extreme (LD=1, LDfrac=0.14)
capable to able (LD=3, LDfrac=0.43)
rare to rare (LD=0, LDfrac=0.00)
utile to useful (LD=5, LDfrac=0.83)
exact to exact (LD=0, LDfrac=0.00)
curieuses to curious (LD=3, LDfrac=0.33)
original to original (LD=0, LDfrac=0.00)
confortable to comfortable (LD=1, LDfrac=0.09)
cousin to cousin (LD=0, LDfrac=0.00)
extraordinaire to extraordinary (LD=2, LDfrac=0.14)
intéressée to interested (LD=4, LDfrac=0.40)
étran

extrêmement to extremely (LD=4, LDfrac=0.36)
jouent to are playing (LD=10, LDfrac=0.91)
dedans to in (LD=5, LDfrac=0.83)
principalement to mainly (LD=11, LDfrac=0.79)
apparemment to apparently (LD=5, LDfrac=0.45)
totalement to totally (LD=5, LDfrac=0.50)
définitivement to finally (LD=11, LDfrac=0.79)
franchement to frankly (LD=7, LDfrac=0.64)
debout to standing (LD=8, LDfrac=1.00)
gratuitement to free (LD=9, LDfrac=0.75)
longtemps to long time (LD=4, LDfrac=0.44)
mieux to better (LD=6, LDfrac=1.00)
gaspiller to waste (LD=6, LDfrac=0.67)
fortement to strongly (LD=8, LDfrac=0.89)
réellement to actually (LD=10, LDfrac=1.00)
construction to construction (LD=0, LDfrac=0.00)
enfance to childhood (LD=9, LDfrac=1.00)
solution to solution (LD=0, LDfrac=0.00)
faute to fault (LD=2, LDfrac=0.40)
victoire to victory (LD=2, LDfrac=0.25)
phénomène to phenomenon (LD=4, LDfrac=0.40)
récent to recent (LD=1, LDfrac=0.17)
contrôle to control (LD=2, LDfrac=0.25)
voir to see (LD=4, LDfrac=1.00)
différence t

recevoir to receive (LD=4, LDfrac=0.50)
vivre to live (LD=2, LDfrac=0.40)
finir to finish (LD=2, LDfrac=0.33)
permettre to allow (LD=9, LDfrac=1.00)
changer to change (LD=1, LDfrac=0.14)
garder to keep (LD=5, LDfrac=0.83)
quitter to leave (LD=6, LDfrac=0.86)
reprendre to resume (LD=6, LDfrac=0.67)
message to message (LD=0, LDfrac=0.00)
argument to argument (LD=0, LDfrac=0.00)
montrer to show (LD=7, LDfrac=1.00)
dents to teeth (LD=3, LDfrac=0.60)
peuvent to can (LD=6, LDfrac=0.86)
sec to dry (LD=3, LDfrac=1.00)
foncées to dark (LD=7, LDfrac=1.00)
par to by (LD=3, LDfrac=1.00)
possédons to own (LD=7, LDfrac=0.78)
samedis to saturdays (LD=5, LDfrac=0.56)
dimanches to sundays (LD=7, LDfrac=0.78)
prends to take (LD=6, LDfrac=1.00)
montres to watches (LD=5, LDfrac=0.71)
lourdes to heavy (LD=7, LDfrac=1.00)
épouse to wife (LD=5, LDfrac=0.83)
stupide to dumb (LD=6, LDfrac=0.86)
médecin to doctor (LD=6, LDfrac=0.86)
horloge to clock (LD=5, LDfrac=0.71)
roue to wheel (LD=4, LDfrac=0.80)
fenêtres

projet to project (LD=1, LDfrac=0.14)
possibilité to possibility (LD=1, LDfrac=0.09)
détail to detail (LD=1, LDfrac=0.17)
rencontrer to meet (LD=8, LDfrac=0.80)
entrer to enter (LD=1, LDfrac=0.17)
résoudre to solve (LD=5, LDfrac=0.62)
protéger to protect (LD=4, LDfrac=0.50)
décrire to describe (LD=3, LDfrac=0.38)
fermer to close (LD=5, LDfrac=0.83)
organiser to organize (LD=2, LDfrac=0.22)
assiette to plate (LD=6, LDfrac=0.75)
abandonner to abandon (LD=3, LDfrac=0.30)
accompagner to accompany (LD=3, LDfrac=0.27)
contacter to contact (LD=2, LDfrac=0.22)
retourner to return (LD=3, LDfrac=0.33)
boule to ball (LD=3, LDfrac=0.60)
méchants to bad guys (LD=7, LDfrac=0.88)
pièce to piece (LD=1, LDfrac=0.20)
coeur to heart (LD=4, LDfrac=0.80)
argent to money (LD=5, LDfrac=0.83)
parfaites to perfect (LD=5, LDfrac=0.56)
couverture to blanket (LD=9, LDfrac=0.90)
rideaux to curtains (LD=7, LDfrac=0.88)
district to district (LD=0, LDfrac=0.00)
travaillent to working (LD=9, LDfrac=0.82)
sage to wise 

In [19]:
print("donedone")

donedone


In [14]:
clean_trans = "i will eat"
g = clean_trans.split(" ")
if len(g) == 2:
    if g[0] in ['to','i','we','you','they','he','she','the',"i'll"]:
        clean_trans = g[1]          
elif len(g) > 2:
    if g[1] in ['will']:
        clean_trans = ' '.join(g[2:len(g)])
    print("string with %i parts detected" % (len(g)))
    
print(clean_trans)

string with 3 parts detected
eat
